# MariaDB Reference Material
<hr>
This reference material is created to complement the RDBMS material taught during the lecture

To test if a connection to the MariaDB can be established.<br>
Change the <b><code>password</code></b> value to your chosen password

In [3]:
# imports mariadb library
import mariadb

try:
    # Instantiate Connection
    conn = mariadb.connect(
        user="root",
        password="",
        host="localhost",
        port=3306)
    print("success")
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")


success


Once you finish working with the database make sure that you close this connection to avoid keeping unused connections open and thus wasting resources. You can close the connection with the close() method:

In [62]:
# Create a cursor object
conn = mariadb.connect(
    user="root",
    password="",
    host="localhost",
    port=3306)
cur  = conn.cursor() 

In [63]:
cur.execute("DROP DATABASE nation_new")

In [64]:
# creating database  
cur.execute("CREATE DATABASE nation_new")

In [65]:
cur.execute("SHOW DATABASES") 
databaseList = cur.fetchall() 

In [66]:
for database in databaseList: 
  print(database) 
# Close Connection
conn.close()

('information_schema',)
('mysql',)
('nation',)
('nation_new',)
('performance_schema',)
('sales',)


In [67]:
# Disable Auto-Commit
#conn.autocommit = False

In [68]:
#cur.execute("DROP DATABASE IF EXISTS nation_new")

In [69]:
#cur.execute("DROP TABLE IF EXISTS PRODUCT")  
#query = """CREATE TABLE PRODUCT (  
#         PRODUCT_ID  CHAR(20) NOT NULL,  
#         price  int(10),  x`
#         PRODUCT_TYPE VARCHAR(64) ) """
  
# To execute the SQL query 
#cur.execute(query)    

<hr>
We will be using the MariaDB <code>context manager</code> from this point onwards and the password was set to <b></b>. 
<br>
Thus you may need to change it to your choosen password.

In [70]:
import mariadb_context as rdbms

Attributes and methods of the `cursor`object

In [71]:
# to view the functions available in the cursor object
with rdbms.MariaDBCursor("", "nation_new") as rdbms_cursor:
    print(dir(rdbms_cursor))

['__class__', '__delattr__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'arraysize', 'buffered', 'callproc', 'close', 'closed', 'connection', 'description', 'execute', 'executemany', 'fetchall', 'fetchmany', 'fetchone', 'fieldcount', 'lastrowid', 'next', 'nextset', 'rowcount', 'rownumber', 'scroll', 'setinputsizes', 'setoutputsize', 'sp_outparams', 'statement', 'warnings']


## Create operations

SQL statements to create the tables

In [72]:
# list of tables
db_tables = []

table_languages = """
    CREATE TABLE languages( language_id INT AUTO_INCREMENT, 
        language VARCHAR(50) NOT NULL,
        PRIMARY KEY(language_id)
);"""

table_continents = """
    CREATE TABLE continents( continent_id INT AUTO_INCREMENT, 
        name VARCHAR(255) NOT NULL,
        PRIMARY KEY(continent_id)
);"""

table_region = """
    CREATE TABLE regions( region_id INT AUTO_INCREMENT, name VARCHAR(100) NOT NULL,
        continent_id INT NOT NULL,
        PRIMARY KEY(region_id),
        FOREIGN KEY(continent_id) 
            REFERENCES continents(continent_id) ON UPDATE CASCADE ON DELETE CASCADE
);"""

table_countries = """
    CREATE TABLE countries ( country_id INT AUTO_INCREMENT, 
        name VARCHAR(50) NOT NULL,
        area DECIMAL(10,2) NOT NULL,
        national_day DATE,
        country_code2 CHAR(2) NOT NULL UNIQUE,
        country_code3 CHAR(3) NOT NULL UNIQUE,
        region_id INT NOT NULL,
        FOREIGN KEY(region_id) 
            REFERENCES regions(region_id) ON UPDATE CASCADE ON DELETE CASCADE,
        PRIMARY KEY(country_id)
);"""

table_country_stats = """
    CREATE TABLE country_stats( country_id INT, year INT, population INT,
        gdp DECIMAL(15,0),
        PRIMARY KEY(country_id, year),
        FOREIGN KEY(country_id)
            REFERENCES countries(country_id) ON UPDATE CASCADE ON DELETE CASCADE
);"""

table_country_languages = """
    CREATE TABLE country_languages( country_id INT, language_id INT, 
        official BOOLEAN NOT NULL,
        PRIMARY KEY(country_id, language_id),
        FOREIGN KEY(country_id) 
            REFERENCES countries(country_id) ON UPDATE CASCADE ON DELETE CASCADE,
        FOREIGN KEY(language_id)
            REFERENCES languages(language_id) ON UPDATE CASCADE ON DELETE CASCADE
);"""

db_tables = [table_languages, table_continents, table_region, table_countries, table_country_stats,
         table_country_languages]

<b>Create</b> the tables using a loop

In [73]:
with rdbms.MariaDBCursor("", "nation_new") as rdbms_cursor:
    for table in db_tables:
        rdbms_cursor.execute(table)
print("Create tables done")

Create tables done


## Update/Insert operations

Prepare the insert statements into a list

In [74]:
# list of insert statements
insert_statements = []

insert_lang = """
    INSERT INTO languages(language) VALUES 
        ("English"), ("Mandarin"), ("Malay"), ("Indonesian"), ("German");
"""

insert_continents = """
    INSERT INTO continents(name) VALUES ("Europe"), ("Americas"), ("Asia"), 
        ("Oceania"), ("Africa");
"""

insert_regions = """
    INSERT INTO regions(name, continent_id) VALUES ("Eastern Europe", 1), ("Central Europe", 1), 
        ("North America", 2), ("South America", 2), ("South East Asia", 3), ("East Asia", 3), 
        ("Central Asia", 3), ("Australasia", 4), ("North Africa", 5), ("South Africa", 5), 
        ("East Africa", 5), ("West Africa", 5), ("Central Africa", 5);
"""

insert_countries = """
    INSERT INTO countries(name, area, national_day, country_code2, country_code3, region_id) VALUES
        ("Singapore", 709.2, "1965-09-09", "SG", "SGP", 5), 
        ("Malaysia", 330803, "1957-09-30", "MY", "MYS", 5), 
        ("Indonesia", 1904569, "1945-09-17", "ID", "IDN", 5), 
        ("United States of America", 3796742, "1776-07-04", "US", "USA", 3), 
        ("Australia", 7692024 , "1901-01-26", "AU", "AUS", 8), 
        ("Germany", 357022 , "1990-10-03", "DE", "DEU", 2), 
        ("China", 9596961 , "1949-10-01", "CN", "CHN", 6), 
        ("Canada", 9984670 , "1867-07-01", "CA", "CAN", 3), 
        ("Switzerland", 41285 , "1891-08-01", "CH", "CHE", 2), 
        ("Nigeria", 923769  , "1960-10-01", "NG", "NGA", 12), 
        ("South Africa", 1221037 , "1994-04-27", "ZA", "ZAF", 10);
"""

insert_country_stats = """
    INSERT INTO country_stats(country_id, year, population, gdp) VALUES
        (1, 2019, 5, 586),
        (2, 2020, 32, 1148),
        (3, 2019, 267, 3740),
        (4, 2019, 328, 22321),
        (5, 2020, 25, 1423),
        (6, 2019, 83, 4444),
        (7, 2019, 1400, 29471),
        (8, 2020, 38, 1971),
        (9, 2019, 8, 548),
        (10, 2020, 204, 1275),
        (11, 2019, 58, 813);
"""

insert_country_languages = """
    INSERT INTO country_languages(country_id, language_id, official) VALUES
        (1, 1, TRUE),
        (1, 2, FALSE),
        (1, 3, FALSE),
        (2, 3, TRUE),
        (2, 1, FALSE),
        (3, 4, TRUE),
        (4, 1, TRUE),
        (5, 1, TRUE),
        (6, 5, TRUE),
        (7, 2, TRUE),
        (8, 1, TRUE),
        (9, 5, TRUE),
        (10, 1, TRUE),
        (11, 1, TRUE);
"""

insert_statements = [insert_lang, insert_continents, insert_regions, insert_countries,
                     insert_country_stats, insert_country_languages]

<b>Insert</b> data into the tables

In [75]:
with rdbms.MariaDBCursor("", "nation_new") as rdbms_cursor:
    for insert_statement in insert_statements:
        rdbms_cursor.execute(insert_statement)

print("Done inserting the records.")

Done inserting the records.


<b>Update</b> a record in the regions table

In [76]:
with rdbms.MariaDBCursor("", "nation_new") as rdbms_cursor:
    rdbms_cursor.execute("UPDATE regions set name='Western Europe' WHERE region_id=1;")

print("Done updating")

Done updating


<b>Changing</b> a table's name

In [77]:
with rdbms.MariaDBCursor("", "nation_new") as rdbms_cursor:
    rdbms_cursor.execute("ALTER TABLE countries CHANGE COLUMN national_day independence_day DATE;")

print("Done altering")

Done altering


## Read operations
Select statements

Select <b>all</b> from table <code>countries</code>

In [78]:
with rdbms.MariaDBCursor("", "nation_new") as rdbms_cursor:
    rdbms_cursor.execute("SELECT * FROM countries;")
    
    rows = rdbms_cursor.fetchall()
    
    # print out the rows retrieved
    for row in rows:
        print(row)

(1, 'Singapore', Decimal('709.20'), datetime.date(1965, 9, 9), 'SG', 'SGP', 5)
(2, 'Malaysia', Decimal('330803.00'), datetime.date(1957, 9, 30), 'MY', 'MYS', 5)
(3, 'Indonesia', Decimal('1904569.00'), datetime.date(1945, 9, 17), 'ID', 'IDN', 5)
(4, 'United States of America', Decimal('3796742.00'), datetime.date(1776, 7, 4), 'US', 'USA', 3)
(5, 'Australia', Decimal('7692024.00'), datetime.date(1901, 1, 26), 'AU', 'AUS', 8)
(6, 'Germany', Decimal('357022.00'), datetime.date(1990, 10, 3), 'DE', 'DEU', 2)
(7, 'China', Decimal('9596961.00'), datetime.date(1949, 10, 1), 'CN', 'CHN', 6)
(8, 'Canada', Decimal('9984670.00'), datetime.date(1867, 7, 1), 'CA', 'CAN', 3)
(9, 'Switzerland', Decimal('41285.00'), datetime.date(1891, 8, 1), 'CH', 'CHE', 2)
(10, 'Nigeria', Decimal('923769.00'), datetime.date(1960, 10, 1), 'NG', 'NGA', 12)
(11, 'South Africa', Decimal('1221037.00'), datetime.date(1994, 4, 27), 'ZA', 'ZAF', 10)


Select <code>name</code> and <code>area</code> columns from the table <code>countries</code> and showing only the <b>first 3 records in ascending order</b> by <code>area</code>.

In [79]:
with rdbms.MariaDBCursor("", "nation_new") as rdbms_cursor:
    rdbms_cursor.execute("SELECT name, area FROM countries ORDER BY area LIMIT 3;")
    
    rows = rdbms_cursor.fetchall()
    
    # print out the rows retrieved
    for row in rows:
        print(row)

('Singapore', Decimal('709.20'))
('Switzerland', Decimal('41285.00'))
('Malaysia', Decimal('330803.00'))


Selecting only the countries with with land masses <b>larger than 3 million square kilometers</b> and ordered by area in <b>descending</b> order

In [80]:
with rdbms.MariaDBCursor("", "nation_new") as rdbms_cursor:
    rdbms_cursor.execute("SELECT * FROM countries WHERE area > 3000000 ORDER BY area DESC;")
    
    rows = rdbms_cursor.fetchall()
    
    # print out the rows retrieved
    for row in rows:
        print(row)

(8, 'Canada', Decimal('9984670.00'), datetime.date(1867, 7, 1), 'CA', 'CAN', 3)
(7, 'China', Decimal('9596961.00'), datetime.date(1949, 10, 1), 'CN', 'CHN', 6)
(5, 'Australia', Decimal('7692024.00'), datetime.date(1901, 1, 26), 'AU', 'AUS', 8)
(4, 'United States of America', Decimal('3796742.00'), datetime.date(1776, 7, 4), 'US', 'USA', 3)


Selecting the <b>names of countries</b> within the African and European regions and ordered by name in ascending order. That means the countries have a `region id` of `1,2,9 to 13`.

In [81]:
with rdbms.MariaDBCursor("", "nation_new") as rdbms_cursor:
    rdbms_cursor.execute("SELECT name FROM countries WHERE region_id in " + \
                         "(1,2) or region_id between 9 and 13 ORDER BY name;")
    
    # using fetching 1 record at a time
    row = rdbms_cursor.fetchone()
    while row is not None:
        print(row)
        row = rdbms_cursor.fetchone()

('Germany',)
('Nigeria',)
('South Africa',)
('Switzerland',)


Selecting the country names and region names from the tables **`countries` and `regions`** where their `region_id` matches and ordered by country names.

In [82]:
with rdbms.MariaDBCursor("", "nation_new") as rdbms_cursor:
    rdbms_cursor.execute("SELECT c.name, r.name FROM countries c INNER JOIN " + \
                         "regions r ON r.region_id = c.region_id ORDER BY c.name;")
    
    # fetch the first 3 records of the returned query
    # change it to fetchall() to see the full list
    rows = rdbms_cursor.fetchmany(3)
    
    # print out the rows retrieved
    for row in rows:
        print(row)

('Australia', 'Australasia')
('Canada', 'North America')
('China', 'East Asia')


Selecting the **only** the countries that speaks `English` from the tables **`countries`, `country_languages` and `languages`** where their `country_id` and `language_id` matches and ordered by country names.

In [83]:
with rdbms.MariaDBCursor("", "nation_new") as rdbms_cursor:
    rdbms_cursor.execute("SELECT c.name, l.language FROM countries c " + \
                         "INNER JOIN country_languages USING (country_id) " + \
                         "INNER JOIN languages l USING (language_id) " +\
                         "WHERE l.language = 'English' ORDER BY c.name;"
                        )
    
    rows = rdbms_cursor.fetchall()
    
    # print out the rows retrieved
    for row in rows:
        print(row)

('Australia', 'English')
('Canada', 'English')
('Malaysia', 'English')
('Nigeria', 'English')
('Singapore', 'English')
('South Africa', 'English')
('United States of America', 'English')


Mixing an aggregate function (`sum`) to show the total landmass areas of the countries within the regions.

In [84]:
with rdbms.MariaDBCursor("", "nation_new") as rdbms_cursor:
    rdbms_cursor.execute("SELECT r.name, sum(area) sum_land FROM countries " + \
                         "INNER JOIN regions r USING (region_id) GROUP BY r.name " + \
                         "ORDER BY sum_land;"
                        )
    
    rows = rdbms_cursor.fetchall()
    
    # print out the rows retrieved
    for row in rows:
        print(row)

('Central Europe', Decimal('398307.00'))
('West Africa', Decimal('923769.00'))
('South Africa', Decimal('1221037.00'))
('South East Asia', Decimal('2236081.20'))
('Australasia', Decimal('7692024.00'))
('East Asia', Decimal('9596961.00'))
('North America', Decimal('13781412.00'))


## Delete operations
<br>
Delete a single record

In [85]:
with rdbms.MariaDBCursor("", "nation_new") as rdbms_cursor:
    rdbms_cursor.execute("DELETE FROM countries WHERE name='South Africa';")
    
    print(f"{rdbms_cursor.rowcount} records(s) deleted" )
    
    # showing the results of the other tables after a record is deleted
    rdbms_cursor.execute("SELECT * FROM country_languages;")
    rows = rdbms_cursor.fetchall()
    print("Table: country_languages")
    # print out the rows retrieved
    for row in rows:
        print(row)
    
    rdbms_cursor.execute("SELECT * FROM country_stats;")
    rows = rdbms_cursor.fetchall()
    print("\n\nTable: country_stats")
    # print out the rows retrieved
    for row in rows:
        print(row)

1 records(s) deleted
Table: country_languages
(1, 1, 1)
(1, 2, 0)
(1, 3, 0)
(2, 1, 0)
(2, 3, 1)
(3, 4, 1)
(4, 1, 1)
(5, 1, 1)
(6, 5, 1)
(7, 2, 1)
(8, 1, 1)
(9, 5, 1)
(10, 1, 1)


Table: country_stats
(1, 2019, 5, Decimal('586'))
(2, 2020, 32, Decimal('1148'))
(3, 2019, 267, Decimal('3740'))
(4, 2019, 328, Decimal('22321'))
(5, 2020, 25, Decimal('1423'))
(6, 2019, 83, Decimal('4444'))
(7, 2019, 1400, Decimal('29471'))
(8, 2020, 38, Decimal('1971'))
(9, 2019, 8, Decimal('548'))
(10, 2020, 204, Decimal('1275'))


How to find the foreign key constraint name of all the tables in the database. For more details [click](https://dev.mysql.com/doc/refman/8.0/en/information-schema-key-column-usage-table.html)

In [86]:
find_foreign_key = """
SELECT
    fk.constraint_name, 
    c.ordinal_position,
    c.table_schema,
    c.table_name,
    c.column_name,
    c.referenced_table_schema,
    c.referenced_table_name,
    c.referenced_column_name
  FROM information_schema.table_constraints fk
  JOIN information_schema.key_column_usage c 
    ON c.constraint_name = fk.constraint_name
  WHERE fk.constraint_type = 'FOREIGN KEY';
"""

with rdbms.MariaDBCursor("", "nation_new") as rdbms_cursor:
    rdbms_cursor.execute(find_foreign_key)
    
    rows = rdbms_cursor.fetchall()
    
    # Note that we only want the foreign key constraint name
    # and that is the first value in the tuple
    for row in rows:
        print(row)

('countries_ibfk_1', 1, 'nation', 'countries', 'region_id', 'nation', 'regions', 'region_id')
('countries_ibfk_1', 1, 'nation', 'countries', 'region_id', 'nation', 'regions', 'region_id')
('country_languages_ibfk_1', 1, 'nation', 'country_languages', 'country_id', 'nation', 'countries', 'country_id')
('country_languages_ibfk_1', 1, 'nation', 'country_languages', 'country_id', 'nation', 'countries', 'country_id')
('country_stats_ibfk_1', 1, 'nation', 'country_stats', 'country_id', 'nation', 'countries', 'country_id')
('country_stats_ibfk_1', 1, 'nation', 'country_stats', 'country_id', 'nation', 'countries', 'country_id')
('regions_ibfk_1', 1, 'nation', 'regions', 'continent_id', 'nation', 'continents', 'continent_id')
('regions_ibfk_1', 1, 'nation', 'regions', 'continent_id', 'nation', 'continents', 'continent_id')
('countries_ibfk_1', 1, 'nation_new', 'countries', 'region_id', 'nation_new', 'regions', 'region_id')
('countries_ibfk_1', 1, 'nation_new', 'countries', 'region_id', 'nation_

Delete all rows from the tables `languages`

In [87]:
with rdbms.MariaDBCursor("", "nation_new") as rdbms_cursor:
    rdbms_cursor.execute("ALTER TABLE country_languages DROP " + \
                         "FOREIGN KEY country_languages_ibfk_2;")
    rdbms_cursor.execute("TRUNCATE TABLE languages;")
    
    print(f"{rdbms_cursor.rowcount} records(s) deleted" )
    
    # showing the results of the other tables after a record is deleted
    rdbms_cursor.execute("SELECT * FROM languages;")
    rows = rdbms_cursor.fetchall()
    
    print("Table: languages")
    if rows:
        for row in rows:
            print(row)
    else:
        print("Table has no records")
    

0 records(s) deleted
Table: languages
Table has no records


Delete the table `languages`

In [88]:
with rdbms.MariaDBCursor("", "nation_new") as rdbms_cursor:
    rdbms_cursor.execute("DROP TABLE languages;")
    
    # showing the results of the other tables after a record is deleted
    rdbms_cursor.execute("SHOW tables;")
    rows = rdbms_cursor.fetchall()
    
    print("All the tables in the database")
    for row in rows:
        print(row)

All the tables in the database
('continents',)
('countries',)
('country_languages',)
('country_stats',)
('regions',)
